### NAAM: CEULEMANS ROBBE ---- KLAS: ISB204B
Je levert een **uitgevoerde notebook** in, dus de **resultaten van de berekeningen staan er ook in**. Resultaten moeten **niet** afgerond worden.

In [61]:
# Zet hier je imports 
# Als je gebruik maakt van eigen routines/functies, zet deze ook hier
from mlxtend.frequent_patterns import fpgrowth, association_rules, apriori
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from mlxtend.preprocessing import TransactionEncoder
from scipy import stats
from scipy.stats import binom ,norm, poisson, chi2,t,ttest_1samp, chisquare
from scipy.special import comb
def bayes(A,B,A_als_B):
    B_als_A = (A*A_als_B)/B
    
    return B_als_A

def totale_kans(kansen : list):
    result = 0
    for x in kansen:
        result += x
    return result
def factor(p,n):
    factorresult = t.ppf((1+p)/2, df=n-1)
    return factorresult
def tinterval(p_of_a,n,x_bar,s):
    result = t.interval(confidence=p_of_a, df=n-1,loc = x_bar, scale=s/math.sqrt(n))
    return result
def binneninterval(x_bar, tinterval :  list):
    result = tinterval[0] < x_bar < tinterval[1]
    print("binnen interval? " ,result)
def relatief_percent(tabel: pd.Series, afronden: bool, aantal_cijfers=0):
    if afronden:
        return (tabel.value_counts(normalize=True) * 100).round(aantal_cijfers)
    else:
        return tabel.value_counts(normalize=True) * 100


In [29]:
# Lees hier je csv file "bevolking.csv" 
# Gebruik kolom 0 als index_col
# Indien je voor een vraag deze dataset nodig hebt, staat dit expliciet vermeld
# Voor de andere vragen heb je dus deze dataset niet nodig
bevolking = pd.read_csv('bevolking.csv' ,index_col=0, sep=';',decimal='.')
bevolking

,age,sex,region,income,married,children,car,fiber,iphone,linux
id,,,,,,,,,,
ID12101,48,FEMALE,INNER_CITY,17546.00,False,1,False,True,False,True
ID12102,40,MALE,TOWN,30085.10,True,3,True,True,True,False
ID12103,51,FEMALE,INNER_CITY,16575.40,True,0,True,True,False,False
ID12104,23,FEMALE,TOWN,20375.40,True,3,False,True,False,False
ID12105,57,FEMALE,RURAL,50576.30,True,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
ID12696,61,FEMALE,INNER_CITY,47025.00,False,2,True,True,True,False
ID12697,30,FEMALE,INNER_CITY,9672.25,True,0,True,True,False,False
ID12698,31,FEMALE,TOWN,15976.30,True,0,True,False,False,True


**Vraag 1**

In [30]:
# Antwoord vraag 1
len(bevolking.loc[(bevolking['car'] == True) & (bevolking['region'] == 'TOWN')]) / len(bevolking.loc[bevolking['region'] == 'TOWN'])

0.5260115606936416

**Vraag 2** 

In [133]:
# Antwoord vraag 2
A = 0.50
B = totale_kans([0.001*0.25,0.002*0.25,0.003*0.5])
A_als_B = 0.003  

bayes(A,B,A_als_B)

0.6666666666666666

**Vraag 3**

In [32]:
# Antwoord vraag 3
p = len(bevolking.loc[bevolking['married'] == True]) / len(bevolking)
# print(p)
1 - binom.cdf(6,10,p)

0.5410611676706898

**Vraag 4**

In [33]:
# Antwoord vraag 4
poisson.cdf(2,10)

0.0027693957155115775

**Vraag 5**

In [41]:
# Antwoord vraag 5
1 - norm.cdf(65000,loc=bevolking.income.mean(),scale=bevolking.income.std())
# bevolking.income.mean()

0.0018348953912493693

**Vraag 6**

In [45]:
# Antwoord vraag 6
a = 0.05
p= 1 -a
n = len(bevolking)
xbar = bevolking.age.mean()
s = bevolking.age.std()
t.interval(confidence=p,df=n-1 , loc=xbar,scale=s/math.sqrt(n))

(41.240785028406115, 43.54921497159389)

**Vraag 7**

In [51]:
# Antwoord vraag 7
a = 0.05
p = 1-a
factor = factor(p,len(bevolking.income))
print()
ttest_1samp(bevolking['income'],26000)


TtestResult(statistic=2.8939943583882823, pvalue=0.003942172845663158, df=599)

**Vraag 8**

In [91]:
# Antwoord vraag 8
a = 0.05
actual = relatief_percent(bevolking.children,afronden=False,aantal_cijfers=0)
list =  actual.tolist()
chisquare(list,[45.0,25.0,20.0,10.0])

Power_divergenceResult(statistic=0.7302469135802462, pvalue=0.8660675516588087)

**Vraag 9**

**Vraag 10**

In [131]:
# Antwoord vraag 10



test = pd.DataFrame({'fiber':bevolking.fiber,'iphone':bevolking.iphone,'car':bevolking.car,'linux':bevolking.linux})
itemsets = fpgrowth(test,min_support=0.2, use_colnames=True)
rules = association_rules(itemsets, metric='confidence',min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(linux),(fiber),0.651667,0.803333,0.540000,0.828645,1.031508,0.016494,1.147711,0.087690
1,(car),(fiber),0.493333,0.803333,0.391667,0.793919,0.988281,-0.004644,0.954317,-0.022869
2,"(car, linux)",(fiber),0.325000,0.803333,0.263333,0.810256,1.008618,0.002250,1.036486,0.012658
3,(iphone),(linux),0.348333,0.651667,0.348333,1.000000,1.534527,0.121336,inf,0.534527
4,(iphone),(fiber),0.348333,0.803333,0.256667,0.736842,0.917231,-0.023161,0.747333,-0.121630
5,"(fiber, iphone)",(linux),0.256667,0.651667,0.256667,1.000000,1.534527,0.089406,inf,0.468610
6,"(iphone, linux)",(fiber),0.348333,0.803333,0.256667,0.736842,0.917231,-0.023161,0.747333,-0.121630
7,(iphone),"(fiber, linux)",0.348333,0.540000,0.256667,0.736842,1.364522,0.068567,1.748000,0.409938
